In [33]:
import os

In [34]:
pwd

'c:\\Users\\PUJA\\OneDrive\\Desktop\\Machine_Learning_Project\\ML_project-_using_MlFlow\\research'

In [35]:
os.chdir("../")

In [36]:
pwd

'c:\\Users\\PUJA\\OneDrive\\Desktop\\Machine_Learning_Project\\ML_project-_using_MlFlow'

In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    db_host: str
    db_user: str
    db_password: str
    db_name: str
    query: str


In [38]:

from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories


In [39]:
# copy this code and paste it into src/mlproject/config/configuration.py

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            db_host=config.db_host,
            db_user=config.db_user,
            db_password=config.db_password,
            db_name=config.db_name,
            query=config.query
        )

        return data_ingestion_config

In [40]:
import os
import urllib.request as request
import csv
from mlProject import logger
from mlProject.utils.common import get_size
import pymysql


In [41]:
# src/mlProject/DataIngestionModule/data_ingestion.py

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def fetch_data_from_sql(self):
        try:
            # Connect to the MySQL database
            with pymysql.connect(
                host=self.config.db_host,
                user=self.config.db_user,
                password=self.config.db_password,
                database=self.config.db_name,
                charset='utf8mb4',
                cursorclass=pymysql.cursors.DictCursor
            ) as conn:
                cursor = conn.cursor()
                cursor.execute(self.config.query)
                data = cursor.fetchall()
                logger.info(f"Data fetched successfully from MySQL database.")
                return data
        except Exception as e:
            logger.error(f"Error fetching data from MySQL database: {str(e)}")
            raise
    
    
    
    def save_data_to_csv(self, data):
        try:
            output_file_path = Path(self.config.root_dir) / "output_data.csv"
            # Ensure the parent directory exists
            os.makedirs(output_file_path.parent, exist_ok=True)

            with open(output_file_path, mode='w', newline='') as csv_file:
                fieldnames = data[0].keys() if data else []
                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

                # Write header
                writer.writeheader()

                # Write data
                writer.writerows(data)

                logger.info(f"Data saved to CSV file: {output_file_path}")
        except Exception as e:
            logger.error(f"Error saving data to CSV file: {str(e)}")
            raise
        
    
    def process_data(self, data):
        self.save_data_to_csv(data)
        # Add any additional processing logic here

    def ingest_data(self):
        data = self.fetch_data_from_sql()
        self.process_data(data)


In [42]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    # Fetch data from SQL
    sql_data = data_ingestion.fetch_data_from_sql()

    # Process and ingest data
    data_ingestion.process_data(sql_data)

except Exception as e:
    raise e

[2024-01-10 17:33:06,882: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-10 17:33:06,890: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-10 17:33:06,911: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-10 17:33:06,919: INFO: common: created directory at: artifacts]
[2024-01-10 17:33:06,920: INFO: common: created directory at: artifacts/data_ingestion]


[2024-01-10 17:33:07,489: INFO: 1774662623: Data fetched successfully from MySQL database.]
[2024-01-10 17:33:07,529: INFO: 1774662623: Data saved to CSV file: artifacts\data_ingestion\output_data.csv]
